## Normal Self Trained Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16

training_set = train_datagen.flow_from_directory(
    '/Users/sagar/Desktop/friends-identify/dataset/training_set',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')

test_set = test_datagen.flow_from_directory(
    '/Users/sagar/Desktop/friends-identify/dataset/test_set',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical')

# Build the CNN Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(training_set.num_classes, activation='softmax')
])

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit(
    training_set,
    steps_per_epoch=training_set.samples // batch_size,
    epochs=20,
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size)

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_set, steps=test_set.samples // batch_size)
print("Test accuracy:", test_accuracy)


In [ ]:
print(training_set.class_indices)
print(test_set.class_indices)

In [ ]:
import numpy as np
import keras.utils as image
test_image = image.load_img('/Users/sagar/Desktop/friends-identify/dataset/angel1.jpg', target_size=(150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)
prediction = ''
if result[0][0] == 1: 
    prediction = 'angel'
elif result[0][1] == 1:
    prediction = 'anup'
else:
    prediction = 'sagar'
print("It most likely is ", prediction)

## Using pre-trained model for better results with low data availability

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 8  # Smaller batch size

# Load training and testing data using generators
training_set = train_datagen.flow_from_directory(
    '/Users/sagar/Desktop/friends-identify/dataset/training_set',
    target_size=(50, 50),
    batch_size=batch_size,
    class_mode='categorical')

test_set = test_datagen.flow_from_directory(
    '/Users/sagar/Desktop/friends-identify/dataset/test_set',
    target_size=(50, 50),
    batch_size=batch_size,
    class_mode='categorical')

# Build the CNN Model with Transfer Learning
base_model = tf.keras.applications.MobileNetV2(input_shape=(50, 50, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(training_set.num_classes, activation='softmax')
])

# Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit(
    training_set,
    steps_per_epoch=training_set.samples // batch_size,
    epochs=40,  # Increase epochs
    validation_data=test_set,
    validation_steps=test_set.samples // batch_size)

# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_set, steps=test_set.samples // batch_size)
print("Test accuracy:", test_accuracy)


In [ ]:
model.save('my_face_recognition_model.h5')

In [ ]:
print(training_set.class_indices)

In [ ]:
import numpy as np
import keras.utils as image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
test_image = image.load_img('/Users/sagar/Desktop/friends-identify/dataset/anup1.jpg', target_size=(50,50))
test_image = image.img_to_array(test_image)
test_image = preprocess_input(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)
print(result*100)
prediction = ''


predictions = model.predict(test_image)

class_labels = training_set.class_indices.keys()
predicted_class_index = np.argmax(predictions)  # Get the index with the highest probability

print("Predicted class index:", predicted_class_index)
print("Predicted class probabilities:", predictions)

if predicted_class_index == 0: 
    prediction = 'angel'
elif predicted_class_index == 1:
    prediction = 'anup'
elif predicted_class_index == 2:
    prediction = 'sagar'
print("It most likely is ", prediction)

## Code to filter and group images into different folder per person

In [ ]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the trained model
model_path = 'my_face_recognition_model.h5'  # Update with your actual model path
model = tf.keras.models.load_model(model_path)

# Directory containing images to predict
input_folder = '/Users/sagar/Desktop/friends-identify/dataset/all'  # Update with your input folder path

# Create an output directory
output_folder = '/Users/sagar/Desktop/friends-identify/dataset/out'  # Update with your output folder path
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through images in the input folder
image_filenames = os.listdir(input_folder)
for image_filename in image_filenames:
    image_path = os.path.join(input_folder, image_filename)
    
    # Load and preprocess the image
    img = load_img(image_path, target_size=(50, 50))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict the class
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions)
    
   # Get the class label based on the highest probability
    class_labels = training_set.class_indices.keys()
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = list(class_labels)[predicted_class_index]
    
    # Create a folder for the predicted class if it doesn't exist
    class_output_folder = os.path.join(output_folder, predicted_class_label)
    if not os.path.exists(class_output_folder):
        os.makedirs(class_output_folder)
    
    # Copy the image to the respective class folder
    shutil.copy(image_path, class_output_folder)

print("Prediction and organization completed.")


## Creates a OpenCV frame and uses the above model to predict person's name who is on the frame

In [ ]:
pip install opencv-python

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Load the trained model
model_path = 'my_face_recognition_model.h5'  # Replace with your actual model path
model = tf.keras.models.load_model(model_path)

# Load the Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define class labels
class_labels = ['angel', 'anup', 'sagar']

# Open a video capture object
cap = cv2.VideoCapture(0)  # 0 for default webcam, or use a video file path

while True:
    ret, frame = cap.read()  # Read a frame from the video feed
    
    if not ret:
        break
    
    # Convert frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces using Haar cascade
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        # Extract the face region
        face_roi = frame[y:y+h, x:x+w]
        
        # Preprocess the face
        resized_face = cv2.resize(face_roi, (50, 50))
        img_array = img_to_array(resized_face)
        img_array = preprocess_input(img_array)
        img_array = np.expand_dims(img_array, axis=0)
        
        # Predict the class probabilities
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        predicted_class_label = class_labels[predicted_class_index]
        
        # Draw bounding box around the face and display predicted class name
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        text = f'Predicted: {predicted_class_label}'
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    cv2.imshow('Video Feed', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

# Release the video capture object and close the windows
cap.release()
cv2.destroyAllWindows()
